In [292]:
import re
import math
import numpy as np
import pandas as pd
from PIL import Image

In [358]:
MAX_BYTES=  100000
def read_str(filename, jump = False):
    byte_list = []
    with open (filename, "rb") as myfile:
        count = 0;
        byte = myfile.read(1)
        byte_list.append('{0:08b}'.format(ord(byte)))
        while count < MAX_BYTES:
            byte = myfile.read(1)
            if (byte == ""):
                break
            else :  
                #ignoring null bytes for now
                if ('00000000' != '{0:08b}'.format(ord(byte))) :
                    byte_list.append('{0:08b}'.format(ord(byte)))
                #print('{0:08b}'.format(ord(byte)))
            count += 1
            #jump ahead to get a more even distribution
            if (jump): myfile.read(10000)
    #print("done")
    return pd.Series(byte_list)

s = read_str('music.m4a')

In [317]:
#counting the frequency of each n-bit grouping 
#b is the binary group
#freq_count is current freq count, as an array where the index is the ith permutation of the group 
#and the value is the frequency
#this is non pure! modifies freq count
def count_frequency(b_list):
    #create new list of 0s 
    freq_count = pd.Series([0 for i in range(int(math.pow(2,n)))])
    #for each bit grouping, increment corresponding entry in freq count
    def inner_count(b):
        freq_count[int(b,2)] += 1 
    b_list.map(lambda x: inner_count(x))
    return freq_count

#freq_count = count_frequency(s)

In [323]:
#convert frequency to relative frequency (on [0,1])
def convert_freq_to_rel_freq(freq_count):
    m = freq_count.max()
    return freq_count.map(lambda x : x * 1.0 / m)

#another option is to just show whether the pattern occurred at all as a binary value
#i.e. 1 if it occurred, zero otherwise
def convert_freq_to_binary(freq_count): 
        return freq_count.map(lambda x : int(x != 0))

#rel_freq_count = convert_freq_to_rel_freq(freq_count)
#print(rel_freq_count)
#bin_freq_count = convert_freq_to_binary(freq_count)
#print(bin_freq_count)

In [324]:
#convert 2 dim frequency vector to 3 dim image matrix
def convert_freq_to_matrix(nrows, freq_count):
    ncols = len(freq_count) /nrows
    mat = np.matrix([[0.0 for i in range(ncols)] for i in range(nrows)])
    print("matrix shape: " + str(mat.shape))
    
    for i, v in enumerate(freq_count):
        row = i / ncols #implicit floor
        col = i - (row * nrows)
        #print("row: " + str(row) + " col: " + str(col) + " v: " + str(v))
        mat[row, col] = v
    return mat


In [364]:
#map image coordinates to blocks from freq matrix
img_height = img_width = 256
block_height = block_width = 16
img = Image.new( 'RGB', (img_height, img_width), "black") # create a new black image
pixels = img.load() # create the pixel map

#convert image x, y coordinate to color for the image
def img_coord_to_color(x, y, freq_mat):
    #converts 
    #print("i: " + str(x/16) + " j: " + str(y/16) + " v:" +  str(freq_mat[x/16, y/16]))
    return freq_mat[x/16, y/16]

def print_image(mat, filename):
    for i in range(img.size[0]):    # for every pixel:
        for j in range(img.size[1]):
            color_val = int(img_coord_to_color(j,i, mat) * 256)
            pixels[i,j] = (color_val, color_val, color_val) # set the colour accordingly
    img.show()
    img.save(filename + ".bmp")


In [368]:
def generate_img(filename, extension, jump = False):
    binary_groups = read_str(filename + extension, jump)
    freq_count = count_frequency(binary_groups)
    #bin_freq_count = convert_freq_to_binary(freq_count)
    rel_freq_count = convert_freq_to_rel_freq(freq_count)
    mat =  convert_freq_to_matrix(16, rel_freq_count)
    print_image(mat, filename)

In [369]:
#add your own files here

#generate_img("loremipsum", ".txt")
#generate_img("216p1",".bin")
#generate_img('music', '.m4a', True)
#generate_img('/usr/bin/vim', '', True)
#generate_img('music3', '.mp3', True)
#generate_img('movie', '.mp4', True)
#generate_img('movie copy', '.bmp', False)
#generate_img('imagejpeg_2', '.jpg', False)
#generate_img('spring2015ACESFormal', '.png', False)
#generate_img('more_text', '.txt', False)

matrix shape: (16, 16)
matrix shape: (16, 16)
matrix shape: (16, 16)
